## Routes

The backend exposes several routes for managing instances and solutions.

- `GET /{PROBLEM_UID}/instances/{INSTANCE_UID}`: Retrieve a specific instance for a problem by its UID.
- `GET /{PROBLEM_UID}/instance_info`: Query instance metadata with pagination and filtering support.
- `GET /{PROBLEM_UID}/instance_schema`: Return the JSON schema of the instance model.
- `GET /{PROBLEM_UID}/instance_info/{INSTANCE_UID}`: Retrieve metadata for a specific instance.
- `GET /{PROBLEM_UID}/problem_info`: Retrieve metadata about the problem, including filters and asset information.
- `POST /{PROBLEM_UID}/instances`: Create a new instance and index it for querying. This is protected by an ApiKey, which needs to be provided in the request header as `api_key`.
- `DELETE /{PROBLEM_UID}/instances/{INSTANCE_UID}`: Delete a specific instance by its UID.

### Assets

Instances can have associated assets (e.g., images or thumbnails). The backend provides endpoints to manage these (optional) assets.
Note that the assets will be served by the nginx server, and these endpoints are just for managing the assets or getting the asset paths.

- `POST /{PROBLEM_UID}/assets/{ASSET_CLASS}/{INSTANCE_UID}`: Upload an asset (e.g., image or thumbnail) for a specific instance. This is protected by an ApiKey, which needs to be provided in the request.
- `GET /{PROBLEM_UID}/assets/{INSTANCE_UID}`: Retrieve all assets associated with a specific instance. The response includes the asset classes and their corresponding file paths.
- `DELETE /{PROBLEM_UID}/assets/{ASSET_CLASS}/{INSTANCE_UID}`: Delete a specific asset for an instance. This is protected by an ApiKey, which needs to be provided in the request.

### Solutions (Optional)

If solutions are configured for a problem class, the backend provides endpoints to manage solutions.

- `GET /{PROBLEM_UID}/solutions/{SOLUTION_UID}`: Retrieve a specific solution by its UID.
- `GET /{PROBLEM_UID}/solution_info/{INSTANCE_UID}`: Retrieve paginated solution metadata for a specific instance.
- `GET /{PROBLEM_UID}/solution_schema`: Retrieve the JSON schema of the solution model.
- `POST /{PROBLEM_UID}/solutions`: Enter a new solution for a specific instance. The solution must reference a valid instance UID. This is protected by an ApiKey, which needs to be provided in the request header as `api_key`.
- `DELETE /{PROBLEM_UID}/solutions/{SOLUTION_UID}`: Delete a specific solution by its UID. This operation also removes the solution from the index. This is protected by an ApiKey, which needs to be provided in the request header as `api_key`.

In [1]:
from pydantic import (
    BaseModel,
    Field,
    NonNegativeFloat,
    NonNegativeInt,
    PositiveFloat,
    PositiveInt,
)


class KnapsackInstance(BaseModel):
    """Pydantic model representing a knapsack problem instance."""

    # Metadata
    instance_uid: str = Field(..., description="The unique identifier of the instance")
    origin: str = Field(default="", description="The origin or source of the instance")

    # Instance statistics
    num_items: PositiveInt = Field(
        ..., description="The number of items available in the instance"
    )
    weight_capacity_ratio: PositiveFloat = Field(
        ...,
        description=(
            "The ratio of the total weight of all items to the knapsack capacity. "
            "Calculated as the sum of item weights divided by knapsack capacity."
        ),
    )
    integral: bool = Field(
        default=False,
        description="Specifies if the capacity, values, and weights are integral.",
    )

    # Instance data
    capacity: NonNegativeFloat | NonNegativeInt = Field(
        ..., description="The total capacity of the knapsack"
    )
    item_values: list[NonNegativeFloat | NonNegativeInt] = Field(
        ..., description="The values assigned to each item"
    )
    item_weights: list[NonNegativeFloat | NonNegativeInt] = Field(
        ..., description="The weights assigned to each item"
    )


class KnapsackSolution(BaseModel):
    """Pydantic model representing a solution to a knapsack problem instance."""

    # Solution metadata
    instance_uid: str = Field(
        ..., description="The unique identifier of the corresponding instance"
    )
    objective: NonNegativeFloat = Field(
        ..., description="The objective value of the solution (e.g., total value)"
    )
    authors: str = Field(..., description="The authors or contributors of the solution")

    # Solution data
    selected_items: list[int] = Field(
        ..., description="Indices of the selected items in the solution"
    )


# Configuration constants for the knapsack problem

# Unique identifier for the problem
PROBLEM_UID = "knapsack"

# Shared attribute name for instances and solutions
INSTANCE_UID_ATTRIBUTE = "instance_uid"

# Schema definitions
INSTANCE_SCHEMA = KnapsackInstance
SOLUTION_SCHEMA = (
    KnapsackSolution  # Optional: Set to None if solutions are not supported
)

# Filtering and sorting configurations
RANGE_FILTERS = [
    "num_items",
    "weight_capacity_ratio",
]  # Fields usable for range filters
BOOLEAN_FILTERS = ["integral"]  # Boolean fields usable for filters
SORT_FIELDS = ["num_items", "weight_capacity_ratio"]  # Fields usable for sorting

# Fields for display purposes in instance overviews
DISPLAY_FIELDS = [
    "instance_uid",
    "num_items",
    "weight_capacity_ratio",
    "integral",
    "origin",
]

# Assets associated with the knapsack problem
ASSETS = {"thumbnail": "png", "image": "png"}

# Solution-specific configurations
SOLUTION_SORT_ATTRIBUTES = [
    "objective"
]  # Fields for sorting solutions. A "-" prefix indicates descending order.
SOLUTION_DISPLAY_FIELDS = ["objective", "authors"]  # Fields to display for solutions


In [2]:
import logging
from typing import Any
from requests import Session
from pydantic import BaseModel

# Configure root logger – adjust level as needed
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger(__name__)

class Connector:
    """
    A simple example connector that logs all HTTP calls made, including the URL, parameters, payload, and responses.
    """
    def __init__(
        self,
        base_url: str,
        problem_uid: str,
        api_key: str | None = None,
    ) -> None:
        self.base_url = base_url.rstrip('/')
        self.problem_uid = problem_uid
        self.session = Session()
        if api_key:
            # automatically include API key on all requests
            self.session.headers.update({'api-key': api_key})

    def _request(
        self,
        method: str,
        path: str,
        params: dict[str, Any] | None = None,
        json: Any = None,
        files: Any = None,
    ) -> dict[str, Any] | str:
        """
        Internal helper to perform an HTTP request and log details.
        """
        url = f"{self.base_url}/{self.problem_uid}{path}"
        logger.info("--> %s %s", method, url)
        if params:
            logger.info("    params=%s", params)
        if json is not None:
            logger.info("    json_payload=%s", json)
        if files:
            logger.info("    files=%s", files)

        response = self.session.request(
            method,
            url,
            params=params,
            json=json,
            files=files,
        )

        logger.info("<-- [%d] %s", response.status_code, response.url)
        text = response.text
        if len(text) > 500:
            logger.info("    response_text=%s... [truncated]", text[:500])
        else:
            logger.info("    response_text=%s", text)

        response.raise_for_status()
        try:
            return response.json()
        except ValueError:
            return text

    def get_instance_schema(self) -> dict[str, Any]:
        """Returns the schema for problem instances."""
        return self._request('GET', '/instance_schema')  # type: ignore

    def get_instance(self, instance_uid: str) -> dict[str, Any]:
        """Fetches a specific problem instance by its UID."""
        return self._request('GET', f'/instances/{instance_uid}')  # type: ignore

    def get_all_instance_info(
        self,
        offset: int = 0,
        limit: int = 100,
        params: dict[str, Any] | None = None,
    ) -> dict[str, Any]:
        """Fetches all problem instances."""
        merged: dict[str, Any] = {'offset': offset, 'limit': limit}
        if params:
            merged |= params
        return self._request('GET', '/instance_info', params=merged)  # type: ignore

    def get_instance_info(self, instance_uid: str) -> dict[str, Any]:
        """Fetches information about a specific problem instance."""
        return self._request('GET', f'/instance_info/{instance_uid}')  # type: ignore

    def get_problem_info(self) -> dict[str, Any]:
        """Fetches information about the problem."""
        return self._request('GET', '/problem_info/')  # type: ignore

    def upload_instance(self, instance: BaseModel) -> dict[str, Any]:
        """Uploads a new problem instance."""
        payload = instance.model_dump(mode='json')
        return self._request('POST', '/instances', json=payload)  # type: ignore

    def delete_instance(self, instance_uid: str) -> dict[str, Any]:
        """Deletes a problem instance by its UID."""
        return self._request('DELETE', f'/instances/{instance_uid}')  # type: ignore

    # Assets
    def get_assets(self, instance_uid: str) -> dict[str, Any]:
        """Fetches all assets for a problem instance."""
        return self._request('GET', f'/assets/{instance_uid}')  # type: ignore

    def upload_asset(
        self,
        instance_uid: str,
        asset_class: str,
        asset_path: str,
    ) -> dict[str, Any]:
        """Uploads an asset for a problem instance."""
        with open(asset_path, 'rb') as f:
            files_data = {'file': f}
            return self._request('POST', f'/assets/{asset_class}/{instance_uid}', files=files_data)  # type: ignore

    def delete_asset(self, instance_uid: str, asset_class: str) -> dict[str, Any]:
        """Deletes a specific asset for a problem instance."""
        return self._request('DELETE', f'/assets/{asset_class}/{instance_uid}')  # type: ignore

    # Solutions
    def get_solution_schema(self) -> dict[str, Any]:
        """Returns the schema for problem solutions."""
        return self._request('GET', '/solution_schema')  # type: ignore

    def get_solution_info(
        self,
        instance_uid: str,
        offset: int = 0,
        limit: int = 100,
    ) -> dict[str, Any]:
        """Fetches the solutions for a specific problem instance."""
        params = {'offset': offset, 'limit': limit}
        return self._request('GET', f'/solution_info/{instance_uid}', params=params)  # type: ignore

    def upload_solution(self, solution: BaseModel) -> dict[str, Any]:
        """Uploads a new solution for a problem instance."""
        payload = solution.model_dump(mode='json')
        return self._request('POST', '/solutions', json=payload)  # type: ignore

    def get_solution(self, solution_uid: str) -> dict[str, Any]:
        """Fetches a specific solution by its UID."""
        return self._request('GET', f'/solutions/{solution_uid}')  # type: ignore

    def delete_solution(self, solution_uid: str) -> dict[str, Any]:
        """Deletes a specific solution for a problem instance."""
        return self._request('DELETE', f'/solutions/{solution_uid}')  # type: ignore
   

    
# For the local example configuration    
connector = Connector(base_url="http://127.0.0.1", problem_uid=PROBLEM_UID, api_key="3456345-456-456")


## Get Problem Metadata

In [3]:
connector.get_problem_info()

2025-05-27 11:25:17,121 INFO --> GET http://127.0.0.1/knapsack/problem_info/
2025-05-27 11:25:17,136 INFO <-- [200] http://127.0.0.1/knapsack/problem_info
2025-05-27 11:25:17,137 INFO     response_text={"problem_uid":"knapsack","range_filters":[{"field_name":"num_items","min_val":11.0,"max_val":1000.0,"problem_uid":"knapsack"},{"field_name":"weight_capacity_ratio","min_val":0.001174003923964717,"max_val":0.9983667695638433,"problem_uid":"knapsack"}],"boolean_filters":["integral"],"sort_fields":["num_items","weight_capacity_ratio"],"display_fields":["instance_uid","num_items","weight_capacity_ratio","integral","origin"],"assets":{"thumbnail":"png","image":"png"}}


{'problem_uid': 'knapsack',
 'range_filters': [{'field_name': 'num_items',
   'min_val': 11.0,
   'max_val': 1000.0,
   'problem_uid': 'knapsack'},
  {'field_name': 'weight_capacity_ratio',
   'min_val': 0.001174003923964717,
   'max_val': 0.9983667695638433,
   'problem_uid': 'knapsack'}],
 'boolean_filters': ['integral'],
 'sort_fields': ['num_items', 'weight_capacity_ratio'],
 'display_fields': ['instance_uid',
  'num_items',
  'weight_capacity_ratio',
  'integral',
  'origin'],
 'assets': {'thumbnail': 'png', 'image': 'png'}}

## Generate and upload some instances

In [4]:
# generate random instances for testing
from random import randint, random
from uuid import uuid4

instances = []
for _ in range(10):
    num_items = randint(10, 1000)
    weight_capacity_ratio = random()
    integral = random() < 0.5
    capacity = random() * 100
    item_values = [random() * 100 for _ in range(num_items)]
    item_weights = [random() * 100 for _ in range(num_items)]
    if integral:
        item_values = [round(v) for v in item_values]
        item_weights = [round(w) for w in item_weights]
        capacity = round(capacity)
    instance = KnapsackInstance(
        instance_uid="test/"+str(uuid4()),
        num_items=num_items,
        weight_capacity_ratio=weight_capacity_ratio,
        integral=integral,
        capacity=capacity,
        item_values=item_values,
        item_weights=item_weights,
    )
    connector.upload_instance(instance)
    


2025-05-27 11:25:17,143 INFO --> POST http://127.0.0.1/knapsack/instances
2025-05-27 11:25:17,143 INFO     json_payload={'instance_uid': 'test/c352d90e-a209-4aa1-b382-07e3db12d8a4', 'origin': '', 'num_items': 710, 'weight_capacity_ratio': 0.05302511655341868, 'integral': True, 'capacity': 30.0, 'item_values': [91.0, 55.0, 99.0, 41.0, 49.0, 36.0, 54.0, 60.0, 67.0, 1.0, 51.0, 4.0, 79.0, 91.0, 98.0, 80.0, 34.0, 41.0, 65.0, 62.0, 91.0, 69.0, 63.0, 49.0, 83.0, 78.0, 41.0, 73.0, 16.0, 74.0, 56.0, 68.0, 56.0, 79.0, 68.0, 51.0, 75.0, 3.0, 73.0, 4.0, 64.0, 74.0, 71.0, 66.0, 62.0, 53.0, 93.0, 65.0, 69.0, 14.0, 54.0, 50.0, 94.0, 66.0, 89.0, 19.0, 32.0, 62.0, 29.0, 61.0, 5.0, 61.0, 21.0, 93.0, 68.0, 50.0, 27.0, 30.0, 48.0, 51.0, 5.0, 87.0, 31.0, 39.0, 33.0, 17.0, 35.0, 11.0, 19.0, 52.0, 88.0, 87.0, 57.0, 33.0, 11.0, 83.0, 9.0, 9.0, 26.0, 67.0, 34.0, 31.0, 55.0, 89.0, 41.0, 20.0, 42.0, 81.0, 72.0, 61.0, 9.0, 86.0, 33.0, 81.0, 28.0, 42.0, 11.0, 61.0, 94.0, 25.0, 83.0, 88.0, 79.0, 31.0, 4.0, 69.0, 5.

In [5]:
all_instances = connector.get_all_instance_info(offset=0, limit=4)
all_instances

2025-05-27 11:25:17,238 INFO --> GET http://127.0.0.1/knapsack/instance_info
2025-05-27 11:25:17,239 INFO     params={'offset': 0, 'limit': 4}
2025-05-27 11:25:17,244 INFO <-- [200] http://127.0.0.1/knapsack/instance_info?offset=0&limit=4
2025-05-27 11:25:17,245 INFO     response_text={"sorted_uids":["test/04168d94-9307-4824-bcdf-b45e62d5ea79","test/f6087360-cf50-4a0b-b296-4a65c657fbc4","test/07692979-6b57-4e19-b220-d1b03e864d09","test/b9214f20-5f5f-45e5-95b4-c46f43e8256b"],"data":{"test/04168d94-9307-4824-bcdf-b45e62d5ea79":{"weight_capacity_ratio":0.3624288186473066,"instance_uid":"test/04168d94-9307-4824-bcdf-b45e62d5ea79","num_items":264,"origin":"","integral":0},"test/f6087360-cf50-4a0b-b296-4a65c657fbc4":{"weight_capacity_ratio":0.9828961048960384,"instance_uid":"test/f... [truncated]


{'sorted_uids': ['test/04168d94-9307-4824-bcdf-b45e62d5ea79',
  'test/f6087360-cf50-4a0b-b296-4a65c657fbc4',
  'test/07692979-6b57-4e19-b220-d1b03e864d09',
  'test/b9214f20-5f5f-45e5-95b4-c46f43e8256b'],
 'data': {'test/04168d94-9307-4824-bcdf-b45e62d5ea79': {'weight_capacity_ratio': 0.3624288186473066,
   'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
   'num_items': 264,
   'origin': '',
   'integral': 0},
  'test/f6087360-cf50-4a0b-b296-4a65c657fbc4': {'weight_capacity_ratio': 0.9828961048960384,
   'instance_uid': 'test/f6087360-cf50-4a0b-b296-4a65c657fbc4',
   'num_items': 144,
   'origin': '',
   'integral': 0},
  'test/07692979-6b57-4e19-b220-d1b03e864d09': {'weight_capacity_ratio': 0.1311399455031409,
   'instance_uid': 'test/07692979-6b57-4e19-b220-d1b03e864d09',
   'num_items': 55,
   'origin': '',
   'integral': 1},
  'test/b9214f20-5f5f-45e5-95b4-c46f43e8256b': {'weight_capacity_ratio': 0.07949394908195195,
   'instance_uid': 'test/b9214f20-5f5f-45e5-95b4-c46f

In [6]:
random_instance_uid = all_instances['sorted_uids'][0]

In [7]:
connector.get_instance_info(random_instance_uid)

2025-05-27 11:25:17,264 INFO --> GET http://127.0.0.1/knapsack/instance_info/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,272 INFO <-- [200] http://127.0.0.1/knapsack/instance_info/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,273 INFO     response_text={"weight_capacity_ratio":0.3624288186473066,"instance_uid":"test/04168d94-9307-4824-bcdf-b45e62d5ea79","num_items":264,"origin":"","integral":0}


{'weight_capacity_ratio': 0.3624288186473066,
 'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
 'num_items': 264,
 'origin': '',
 'integral': 0}

In [8]:
connector.get_instance(random_instance_uid)

2025-05-27 11:25:17,281 INFO --> GET http://127.0.0.1/knapsack/instances/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,289 INFO <-- [200] http://127.0.0.1/knapsack/instances/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,290 INFO     response_text={"instance_uid":"test/04168d94-9307-4824-bcdf-b45e62d5ea79","origin":"","num_items":264,"weight_capacity_ratio":0.3624288186473066,"integral":false,"capacity":50.77823662739207,"item_values":[11.47169459324372,16.24368535341195,11.135907388842492,38.904318593348165,96.65672658755959,98.74899623847747,92.4780096275241,96.395516514061,90.17070856343516,45.02778602763004,49.982329226643266,36.06335165215564,88.00319425607611,70.17680112271637,29.44651940767319,34.98589778265935,87.85296029989179,30... [truncated]


{'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
 'origin': '',
 'num_items': 264,
 'weight_capacity_ratio': 0.3624288186473066,
 'integral': False,
 'capacity': 50.77823662739207,
 'item_values': [11.47169459324372,
  16.24368535341195,
  11.135907388842492,
  38.904318593348165,
  96.65672658755959,
  98.74899623847747,
  92.4780096275241,
  96.395516514061,
  90.17070856343516,
  45.02778602763004,
  49.982329226643266,
  36.06335165215564,
  88.00319425607611,
  70.17680112271637,
  29.44651940767319,
  34.98589778265935,
  87.85296029989179,
  30.498206303492246,
  50.7834114817285,
  14.503041505467317,
  98.16041637797179,
  83.86567668593713,
  81.76197361125591,
  52.365610370073036,
  83.99034756394754,
  34.27381790534986,
  45.57456912983091,
  25.518052731829943,
  55.53294941773645,
  71.83370187179902,
  43.318503051611536,
  49.71449082267026,
  56.582803899791635,
  39.61433967799343,
  42.389695426542275,
  65.93055636487655,
  29.856792417043465,
  28.22

## Assets

In [9]:
# create random png via matplotlib
import matplotlib.pyplot as plt
def create_random_png():
    """Generates a random PNG image."""
    data = [[random() for _ in range(10)] for _ in range(10)]
    plt.imshow(data, cmap='viridis', interpolation='nearest')
    plt.axis('off')
    plt.savefig('random_image.png', format='png', bbox_inches='tight', pad_inches=0)
    plt.close()

create_random_png()

connector.upload_asset(
    instance_uid=random_instance_uid,
    asset_class="thumbnail",
    asset_path="random_image.png"
)

2025-05-27 11:25:17,644 INFO --> POST http://127.0.0.1/knapsack/assets/thumbnail/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,644 INFO     files={'file': <_io.BufferedReader name='random_image.png'>}
2025-05-27 11:25:17,653 INFO <-- [200] http://127.0.0.1/knapsack/assets/thumbnail/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,654 INFO     response_text=null


In [10]:
connector.get_assets(random_instance_uid)

2025-05-27 11:25:17,657 INFO --> GET http://127.0.0.1/knapsack/assets/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,670 INFO <-- [200] http://127.0.0.1/knapsack/assets/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,671 INFO     response_text={"thumbnail":"http://127.0.0.1:80/static/knapsack/assets/thumbnail/test/04168d94-9307-4824-bcdf-b45e62d5ea79.png"}


{'thumbnail': 'http://127.0.0.1:80/static/knapsack/assets/thumbnail/test/04168d94-9307-4824-bcdf-b45e62d5ea79.png'}

In [11]:
connector.delete_asset(
    instance_uid=random_instance_uid,
    asset_class="thumbnail"
)

2025-05-27 11:25:17,675 INFO --> DELETE http://127.0.0.1/knapsack/assets/thumbnail/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,678 INFO <-- [200] http://127.0.0.1/knapsack/assets/thumbnail/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,678 INFO     response_text=null


In [12]:
connector.get_assets(random_instance_uid)

2025-05-27 11:25:17,682 INFO --> GET http://127.0.0.1/knapsack/assets/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,685 INFO <-- [200] http://127.0.0.1/knapsack/assets/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,685 INFO     response_text={}


{}

## Solutions

In [13]:
connector.get_solution_schema()

2025-05-27 11:25:17,690 INFO --> GET http://127.0.0.1/knapsack/solution_schema
2025-05-27 11:25:17,698 INFO <-- [200] http://127.0.0.1/knapsack/solution_schema
2025-05-27 11:25:17,698 INFO     response_text={"description":"Pydantic model representing a solution to a knapsack problem instance.","properties":{"instance_uid":{"description":"The unique identifier of the corresponding instance","title":"Instance Uid","type":"string"},"objective":{"description":"The objective value of the solution (e.g., total value)","minimum":0,"title":"Objective","type":"number"},"authors":{"description":"The authors or contributors of the solution","title":"Authors","type":"string"},"selected_items":{"description":"I... [truncated]


{'description': 'Pydantic model representing a solution to a knapsack problem instance.',
 'properties': {'instance_uid': {'description': 'The unique identifier of the corresponding instance',
   'title': 'Instance Uid',
   'type': 'string'},
  'objective': {'description': 'The objective value of the solution (e.g., total value)',
   'minimum': 0,
   'title': 'Objective',
   'type': 'number'},
  'authors': {'description': 'The authors or contributors of the solution',
   'title': 'Authors',
   'type': 'string'},
  'selected_items': {'description': 'Indices of the selected items in the solution',
   'items': {'type': 'integer'},
   'title': 'Selected Items',
   'type': 'array'}},
 'required': ['instance_uid', 'objective', 'authors', 'selected_items'],
 'title': 'KnapsackSolution',
 'type': 'object'}

In [14]:
connector.get_solution_info(
    instance_uid=random_instance_uid
)

2025-05-27 11:25:17,702 INFO --> GET http://127.0.0.1/knapsack/solution_info/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,702 INFO     params={'offset': 0, 'limit': 100}
2025-05-27 11:25:17,708 INFO <-- [200] http://127.0.0.1/knapsack/solution_info/test/04168d94-9307-4824-bcdf-b45e62d5ea79?offset=0&limit=100
2025-05-27 11:25:17,708 INFO     response_text={"items":[],"offset":0,"limit":100,"total":0}


{'items': [], 'offset': 0, 'limit': 100, 'total': 0}

In [15]:
connector.upload_solution(
    KnapsackSolution(
        instance_uid=random_instance_uid,
        objective=100.0,
        authors="John Doe",
        selected_items=[0, 1, 2]
    )
)

2025-05-27 11:25:17,719 INFO --> POST http://127.0.0.1/knapsack/solutions
2025-05-27 11:25:17,720 INFO     json_payload={'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79', 'objective': 100.0, 'authors': 'John Doe', 'selected_items': [0, 1, 2]}
2025-05-27 11:25:17,737 INFO <-- [200] http://127.0.0.1/knapsack/solutions
2025-05-27 11:25:17,738 INFO     response_text=null


In [16]:
solution_info = connector.get_solution_info(
    instance_uid=random_instance_uid
)
solution_info

2025-05-27 11:25:17,753 INFO --> GET http://127.0.0.1/knapsack/solution_info/test/04168d94-9307-4824-bcdf-b45e62d5ea79
2025-05-27 11:25:17,754 INFO     params={'offset': 0, 'limit': 100}
2025-05-27 11:25:17,762 INFO <-- [200] http://127.0.0.1/knapsack/solution_info/test/04168d94-9307-4824-bcdf-b45e62d5ea79?offset=0&limit=100
2025-05-27 11:25:17,762 INFO     response_text={"items":[{"objective":100.0,"solution_uid":"test/04168d94-9307-4824-bcdf-b45e62d5ea79/a8d100c21cd5b653cd831f2c846817b0","authors":"John Doe","instance_uid":"test/04168d94-9307-4824-bcdf-b45e62d5ea79"}],"offset":0,"limit":100,"total":1}


{'items': [{'objective': 100.0,
   'solution_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79/a8d100c21cd5b653cd831f2c846817b0',
   'authors': 'John Doe',
   'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79'}],
 'offset': 0,
 'limit': 100,
 'total': 1}

In [17]:
random_solution_uid = solution_info['items'][0]['solution_uid']

In [18]:
connector.get_solution(solution_uid=random_solution_uid)

2025-05-27 11:25:17,768 INFO --> GET http://127.0.0.1/knapsack/solutions/test/04168d94-9307-4824-bcdf-b45e62d5ea79/a8d100c21cd5b653cd831f2c846817b0
2025-05-27 11:25:17,772 INFO <-- [200] http://127.0.0.1/knapsack/solutions/test/04168d94-9307-4824-bcdf-b45e62d5ea79/a8d100c21cd5b653cd831f2c846817b0
2025-05-27 11:25:17,773 INFO     response_text={"instance_uid":"test/04168d94-9307-4824-bcdf-b45e62d5ea79","objective":100.0,"authors":"John Doe","selected_items":[0,1,2]}


{'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
 'objective': 100.0,
 'authors': 'John Doe',
 'selected_items': [0, 1, 2]}

In [19]:
connector.delete_solution(solution_uid=random_solution_uid)

2025-05-27 11:25:17,780 INFO --> DELETE http://127.0.0.1/knapsack/solutions/test/04168d94-9307-4824-bcdf-b45e62d5ea79/a8d100c21cd5b653cd831f2c846817b0
2025-05-27 11:25:17,785 INFO <-- [200] http://127.0.0.1/knapsack/solutions/test/04168d94-9307-4824-bcdf-b45e62d5ea79/a8d100c21cd5b653cd831f2c846817b0
2025-05-27 11:25:17,785 INFO     response_text=null
